In [14]:
import sys
from importlib import reload
sys.path.insert(0, "..")
import utils.misc as misc
import distributed
import distributed.sharded_matrix
reload(distributed.sharded_matrix)
from distributed.sharded_matrix import ShardedMatrix, ShardedSymmetricMatrix
from distributed import kernel, matmul, bcd
import numpy as np
import scipy.linalg
import pywren
reload(pywren)
import math
import itertools
import time
import cloudpickle
import importlib
import sys
from sklearn.datasets import fetch_mldata
from itertools import groupby
from operator import itemgetter
from sklearn.kernel_approximation import RBFSampler, Nystroem
from sklearn import metrics
import sys
import boto3
import concurrent.futures as fs
reload(boto3)
from utils import linalg
from utils.misc import rreload
import scipy.linalg 


In [2]:
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [ ]:
%time X_test = np.load("/home/ubuntu/imagenet-data/X_test_lift_3_3_pool_12_12_patch_stride_1.npy")
np.random.seed(0)
idxs_test = np.random.choice(X_test.shape[0], X_test.shape[0], replace=False)
np.save("test_scramble_idxs", idxs_test)
X_test = X_test.reshape(X_test.shape[0],-1)
%time X_test = X_test[idxs_test, :]
labels_test = np.load("/home/ubuntu/imagenet-data/imagenet_test_labels.npy")
labels_test = labels_test[idxs_test]


CPU times: user 548 ms, sys: 3.37 s, total: 3.92 s
Wall time: 29.9 s
CPU times: user 440 ms, sys: 1.12 s, total: 1.56 s
Wall time: 1.55 s


In [20]:
%time X_train_sharded = ShardedMatrix("X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017",\
                                shard_size_0=4096,\
                                bucket="vaishaalpywrenlinalg",\
                                n_jobs=32)

CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 168 ms


In [18]:
%time X_test_sharded = ShardedMatrix("X_test_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017",\
                                shard_size_0=4096,\
                                bucket="vaishaalpywrenlinalg",\
                                reshard=False,\
                                n_jobs=32)

CPU times: user 68 ms, sys: 0 ns, total: 68 ms
Wall time: 123 ms


In [16]:
pwex = pywren.default_executor()

In [183]:
%time XXT_test_sharded = matmul.compute_XYT_pywren(pwex, X=X_train_sharded, Y=X_test_sharded, tasks_per_job=4)

Submitting job for chunk 0 in axis 0 for chunk 0 in axis 1
Submitting job for chunk 1 in axis 0 for chunk 0 in axis 1
Waiting job for chunk 0 in axis 0 for chunk 0 in axis 1
Waiting job for chunk 1 in axis 0 for chunk 0 in axis 1
CPU times: user 1min 59s, sys: 10.3 s, total: 2min 9s
Wall time: 4min 17s


In [262]:
XXT_test_sharded.shape

(1281167, 50000)

In [ ]:
reload(kernel)
%time K_test_sharded = kernel.compute_rbf_kernel_pywren(pwex,\
                                                        XXT_test_sharded,\
                                                        X_test_sharded,\
                                                        X_train_sharded,\
                                                        gamma=1e-1,\
                                                        tasks_per_job=4,\
                                                        num_features=X_test.shape[1])

[0]
Submitting job for chunk 0
Waiting for chunk 0


In [264]:
K_test_sharded

In [236]:
sq_norms_train = matmul.compute_sq_norms_pywren(pwex, X_train_sharded)
sq_norms_test = matmul.compute_sq_norms_pywren(pwex, X_test_sharded)

[0]
Submitting job for chunk 0
Waiting for chunk 0
[0]
Submitting job for chunk 0
Waiting for chunk 0


In [14]:
%time XXT_test_sharded = ShardedMatrix("XYT(X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017, X_test_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017)",\
                                        bucket="vaishaalpywrenlinalg")

CPU times: user 68 ms, sys: 4 ms, total: 72 ms
Wall time: 164 ms


In [2]:
K_test_sharded = ShardedMatrix("rbf(XYT(X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017, X_test_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017), 0.1)",\
                                        bucket="vaishaalpywrenlinalg")

In [9]:
K_train_sharded = ShardedSymmetricMatrix("rbf(XXT(X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_blocks_no_normalize_04.22.2017, 1.0)",
                                         bucket="vaishaalpywrenlinalg")

In [7]:
model = np.load("/home/ubuntu/models/model_3_3_pool_12_12_patch_stride_1_gamma_1e1_reg_1e-1/model_iter_158_train_acc_0.9691905895172136_next_block_acc_0.14152410575427682_norm_5503.890858004476.npy")

In [8]:
train_preds = np.load("/home/ubuntu/models/model_3_3_pool_12_12_patch_stride_1_gamma_1e1_reg_1e-1/yhat_iter_158_train_acc_0.9691905895172136_next_block_acc_0.14152410575427682_norm_5503.890858004476.npy")

In [11]:
idxs = np.load("/home/ubuntu/imagenet-data/scramble_idxs.npy")
labels_train = np.load("/home/ubuntu/imagenet-data/imagenet_train_labels.npy")[idxs]

In [12]:
y_pred_test = np.zeros((K_test_sharded.shape[1], 1000))

In [ ]:
%time futures = bcd.fast_kernel_column_block_async(K_test_sharded, col_blocks=K_test_sharded._block_idxs(1), mmap_loc="/dev/shm/K_test_block", dtype="float32")
%time fs.wait(futures)

%time K_test = bcd.load_mmap("/dev/shm/K_test_block", K_test_sharded.shape, "float32")

CPU times: user 8 ms, sys: 176 ms, total: 184 ms
Wall time: 210 ms


In [7]:
K_test.shape

(1281167, 50000)

In [9]:
model = np.load("/home/ubuntu/model_iter_29_train_acc_0.48995017823593645_next_block_acc_0.212676426755501_norm_1790426.3223825234.npy")

In [10]:
model = model.astype('float32')
%time y_test_pred = K_test.T.dot(model)

CPU times: user 39min 41s, sys: 1h 37min 29s, total: 2h 17min 10s
Wall time: 2min 9s


(50000, 1000)

In [13]:
metrics.accuracy_score(np.argmax(y_test_pred, axis=1), labels_test)

0.16832

In [22]:
%time futures = bcd.fast_kernel_column_block_async(X_train_sharded, col_blocks=X_train_sharded._block_idxs(1), mmap_loc="/dev/shm/X_train", dtype="float32")
%time fs.wait(futures)
%time X_train = bcd.load_mmap(*futures[0].result())

CPU times: user 32 ms, sys: 124 ms, total: 156 ms
Wall time: 152 ms
CPU times: user 36 ms, sys: 12 ms, total: 48 ms
Wall time: 2min 48s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 879 µs


In [256]:
%time futures = bcd.fast_kernel_column_block_async(XXT_test_sharded, col_blocks=[1], mmap_loc="/dev/shm/XXT_block", dtype="float32")
%time fs.wait(futures)
%time XXT_test_column = bcd.load_mmap(*futures[0].result())

CPU times: user 60 ms, sys: 2.96 s, total: 3.02 s
Wall time: 3 s
CPU times: user 200 ms, sys: 20 ms, total: 220 ms
Wall time: 36.6 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1 ms


In [258]:
XXT_test_column

memmap([[  8488.47460938,  14227.68945312,  10847.29492188, ...,
         11020.09082031,   9233.09472656,   7883.08789062],
       [  1973.03588867,   4449.17724609,   2513.03173828, ...,
          2583.4921875 ,   2198.08740234,   1871.6809082 ],
       [  8444.09863281,  11348.98046875,   9798.9921875 , ...,
         10217.67578125,   8411.75683594,   7889.43603516],
       ..., 
       [  2482.96923828,   3555.28881836,   2441.30981445, ...,
          3163.54101562,   2291.53051758,   2629.94335938],
       [  8063.69970703,  15156.08496094,  10762.21386719, ...,
         12204.74902344,   9327.99511719,   7182.95849609],
       [  6105.15478516,  11927.34277344,   7079.953125  , ...,
          8142.71777344,   6466.66015625,   5025.96826172]], dtype=float32)

In [60]:
np.max(np.abs(XXT_test_column[:, :500] - XXT_test_column_local))

memmap(0.015625, dtype=float32)

In [113]:
%time XXT_test_column_local = X_train.dot(X_test[.T)

CPU times: user 3h 4min 56s, sys: 1h 35min 4s, total: 4h 40min
Wall time: 4min 40s


In [ ]:
X_test_sharded.get_block(1,)

In [259]:
XXT_test_column[:, :4096]

memmap([[  8488.47460938,  14227.68945312,  10847.29492188, ...,
         11020.09082031,   9233.09472656,   7883.08789062],
       [  1973.03588867,   4449.17724609,   2513.03173828, ...,
          2583.4921875 ,   2198.08740234,   1871.6809082 ],
       [  8444.09863281,  11348.98046875,   9798.9921875 , ...,
         10217.67578125,   8411.75683594,   7889.43603516],
       ..., 
       [  2482.96923828,   3555.28881836,   2441.30981445, ...,
          3163.54101562,   2291.53051758,   2629.94335938],
       [  8063.69970703,  15156.08496094,  10762.21386719, ...,
         12204.74902344,   9327.99511719,   7182.95849609],
       [  6105.15478516,  11927.34277344,   7079.953125  , ...,
          8142.71777344,   6466.66015625,   5025.96826172]], dtype=float32)

In [205]:
np.linalg.norm(XXT_test_column_local[:, 8192:] - XXT_test_column)

0.0

In [260]:
XXT_test_column_local[:4096, 4096:8192]

array([[  8488.47460938,  14227.68945312,  10847.29492188, ...,
         11020.09082031,   9233.09472656,   7883.08789062],
       [  1973.03588867,   4449.17724609,   2513.03173828, ...,
          2583.4921875 ,   2198.08740234,   1871.6809082 ],
       [  8444.09863281,  11348.98046875,   9798.9921875 , ...,
         10217.67578125,   8411.75683594,   7889.43603516],
       ..., 
       [  8497.02832031,  12133.9765625 ,   8625.77636719, ...,
         10254.43359375,   8831.73828125,   8116.00292969],
       [  7085.04541016,   7805.40917969,   8511.59960938, ...,
          9610.3203125 ,   6687.92382812,   5910.41064453],
       [  2750.84326172,   3047.60620117,   3030.29443359, ...,
          3268.57324219,   2654.40771484,   2605.32519531]], dtype=float32)

In [163]:
reload(matmul)
matmul.compute_xyt_block(XXT_test_sharded, X_train_sharded, X_test_sharded, block_pair_0=(0,1)) 

XYT_block shape (4096, 4096)


array([[  8488.47460938,  14227.68945312,  10847.29492188, ...,
         11020.09082031,   9233.09472656,   7883.08789062],
       [  1973.03588867,   4449.17724609,   2513.03173828, ...,
          2583.4921875 ,   2198.08740234,   1871.6809082 ],
       [  8444.09863281,  11348.98046875,   9798.9921875 , ...,
         10217.67578125,   8411.75683594,   7889.43603516],
       ..., 
       [  8497.02832031,  12133.9765625 ,   8625.77636719, ...,
         10254.43359375,   8831.73828125,   8116.00292969],
       [  7085.04541016,   7805.40917969,   8511.59960938, ...,
          9610.3203125 ,   6687.92382812,   5910.41064453],
       [  2750.84326172,   3047.60620117,   3030.29443359, ...,
          3268.57324219,   2654.40771484,   2605.32519531]], dtype=float32)

In [185]:
XXT_test_sharded.get_block(0,1)

array([[  8488.47460938,  14227.68945312,  10847.29492188, ...,
         11020.09082031,   9233.09472656,   7883.08789062],
       [  1973.03588867,   4449.17724609,   2513.03173828, ...,
          2583.4921875 ,   2198.08740234,   1871.6809082 ],
       [  8444.09863281,  11348.98046875,   9798.9921875 , ...,
         10217.67578125,   8411.75683594,   7889.43603516],
       ..., 
       [  8497.02832031,  12133.9765625 ,   8625.77636719, ...,
         10254.43359375,   8831.73828125,   8116.00292969],
       [  7085.04541016,   7805.40917969,   8511.59960938, ...,
          9610.3203125 ,   6687.92382812,   5910.41064453],
       [  2750.84326172,   3047.60620117,   3030.29443359, ...,
          3268.57324219,   2654.40771484,   2605.32519531]], dtype=float32)

In [80]:
np.linalg.norm(XXT_test_column[:, 4094] - XXT_test_column_local[:, 4094] )

0.0

In [261]:
XXT_test_column

memmap([[  8488.47460938,  14227.68945312,  10847.29492188, ...,
         11020.09082031,   9233.09472656,   7883.08789062],
       [  1973.03588867,   4449.17724609,   2513.03173828, ...,
          2583.4921875 ,   2198.08740234,   1871.6809082 ],
       [  8444.09863281,  11348.98046875,   9798.9921875 , ...,
         10217.67578125,   8411.75683594,   7889.43603516],
       ..., 
       [  2482.96923828,   3555.28881836,   2441.30981445, ...,
          3163.54101562,   2291.53051758,   2629.94335938],
       [  8063.69970703,  15156.08496094,  10762.21386719, ...,
         12204.74902344,   9327.99511719,   7182.95849609],
       [  6105.15478516,  11927.34277344,   7079.953125  , ...,
          8142.71777344,   6466.66015625,   5025.96826172]], dtype=float32)

In [12]:
%time X_train = bcd.load_mmap(*futures[0].result())

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 449 µs


In [19]:
X_train[:4096, :] - X_train_sharded.get_block(0,0)

memmap([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [16]:
X_train.dot(X_test)

array([[  1.96602806e-01,   2.20547840e-01,   1.81348190e-01, ...,
          7.06523836e-01,   8.45599055e-01,   5.61899185e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.34917676e-01,   1.07508719e+00,   1.25146270e+00],
       [  6.73569888e-02,   4.77466770e-02,   4.98547740e-02, ...,
          3.86966228e-01,   7.00732410e-01,   6.33941472e-01],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.50087309e+00,   2.40811825e+00,   2.04348946e+00],
       [  6.10646643e-02,   4.51439694e-02,   6.54195028e-04, ...,
          1.14956582e+00,   6.02480948e-01,   6.63885534e-01],
       [  2.00905767e-03,   5.96098304e-02,   1.14108261e-03, ...,
          1.10903367e-01,   1.67341560e-01,   3.45374405e-01]], dtype=float32)

In [35]:
print("Accuracy is", metrics.accuracy_score(np.argmax(y_pred_test, axis=1), labels_test[4096:8192]))

Accuracy is 0.00439453125


In [36]:
y_pred_test

array([[ 0.15177582,  0.32212979, -0.70441687, ..., -0.9280941 ,
        -0.0218157 ,  0.04977524],
       [-0.68600962,  0.46225398,  0.29152378, ...,  1.02202446,
        -0.18173571, -0.7781125 ],
       [ 0.411547  ,  0.20195045, -0.02920963, ..., -0.4530658 ,
        -0.09862631,  0.46372228],
       ..., 
       [-1.69573461,  1.59853628,  1.4744695 , ...,  0.8763674 ,
        -0.53255569, -0.73806692],
       [ 0.16199264,  0.66496543, -0.05094174, ..., -0.4790934 ,
        -0.34181992,  0.35255426],
       [-0.69530962, -0.04767478, -0.30211379, ...,  0.48934169,
        -0.2773971 , -0.26059168]])

In [5]:
K_test_sharded._block_idxs(1)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [161]:
np.sum(np.argmax(y_test_pred_0, axis=1) == 782)

2

In [132]:
futures = bcd.fast_kernel_column_block_async(K_train_sharded, col_blocks=[300], mmap_loc="/dev/shm/test")
fs.wait(futures)
K_train_block_1 = bcd.load_mmap(*futures[0].result())

In [134]:
%time y_train_pred_0 = K_train_block_1.T.dot(model)

CPU times: user 9min 36s, sys: 44min 50s, total: 54min 27s
Wall time: 52.6 s


In [135]:
metrics.accuracy_score(np.argmax(y_train_pred_0, axis=1), labels_train[300*4096:301*4096])

0.051025390625

In [87]:
metrics.accuracy_score(np.argmax(model[:4096], axis=1), labels_train[:4096])

0.999267578125

In [81]:
np.argmax(model, axis=1)

array([955, 978,  98, ...,   0,   0,   0])

In [133]:
model

array([[ 0.06116991,  0.23227182, -0.6894666 , ...,  0.69814202,
        -0.75120418, -1.04630017],
       [ 1.38090658, -1.13501859,  0.57325702, ...,  0.36937889,
         0.45910894,  1.28379234],
       [-1.5526573 ,  0.10704586,  0.29934353, ...,  0.9878986 ,
        -0.31383154, -0.17818733],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [91]:
y_pred

NameError: name 'y_pred' is not defined

In [144]:
X_test_sharded.get_block(1,0)

array([[ 0.17595319,  0.00642153,  0.01951889, ...,  0.52070808,
         0.99553257,  0.64188957],
       [ 0.01254465,  0.02918571,  0.14533278, ...,  1.9559927 ,
         1.70751333,  1.66638935],
       [ 0.20669395,  0.11355925,  0.17321257, ...,  0.71780199,
         0.84639919,  1.01430058],
       ..., 
       [ 0.0700969 ,  0.05763057,  0.20575623, ...,  1.16097128,
         1.37094891,  2.80404854],
       [ 0.07006628,  0.02370552,  0.10204141, ...,  1.27435827,
         1.39967942,  1.25729001],
       [ 0.00476875,  0.03553968,  0.01166455, ...,  0.50728744,
         1.09930623,  0.89966625]], dtype=float32)